In [ ]:
import ipywidgets as widgets
from IPython.display import display


import os
#import base64
#import requests
#import subprocess
#import mlflow
import ipywidgets as widgets
from IPython.display import display
from matplotlib.animation import FuncAnimation


# load style
#style = widgets.HTML("<style> .blink { animation: blinker 1.5s linear infinite; } @keyframes blinker { 50% {opacity: 0;}}</style>")
#display(style)

style = widgets.HTML("""
    <style>
        .border-left-success {border-left: .25rem solid #1cc88a !important;padding-left: 5px;} 
        .border-left-ready {border-left: .25rem solid #A6A6A6 !important;padding-left: 5px;}
        .substep {margin-left: 50px;}
    </style>""")
display(style)

## load image
with open('images/loading.gif', 'rb') as f:
    img = f.read()
loading_image = widgets.Image(value=img, width=20,height=20)
with open('images/success.png', 'rb') as f:
    img = f.read()
sucess_image = widgets.Image(value=img, width=20,height=20)
with open('images/ready.png', 'rb') as f:
    img = f.read()
ready_image = widgets.Image(value=img, width=20,height=20)


# valiable
domain = 'ua.ezm.host'
mlflow_username = 'leejj881229'
mlflow_password = 'hpinvent!23'
predictor_image = 'marketplace.us1.greenlake-hpe.com/ezmeral/ezkf/qna-vectorstore:v1.3.0-e658264'

username = os.environ['NB_PREFIX'].split('/')[2]
password = 'hpinvent!23'

llm_image = 'marketplace.us1.greenlake-hpe.com/ezmeral/ezkf/qna-llm:v1.3.0-e658264'
transformer_image = 'marketplace.us1.greenlake-hpe.com/ezmeral/ezkf/qna-transformer:v1.3.0-f7315d3'


# input config
display(widgets.HTML("<h3>Please enter information</h3>"))

domain_input = widgets.Text(description='Domain:', value=domain)
username_input = widgets.Text(description='Username:', value=username)
password_input = widgets.Password(description='Password:',value=password)
submit_button = widgets.Button(description='Submit')
config_success_message = widgets.Output()
# Set margin on the submit button
submit_button.layout.margin = '20px 0 20px 0'

def submit_button_clicked(b):
    global domain, mlflow_username, mlflow_password
    domain = domain_input.value
    mlflow_username = username_input.value
    mlflow_password = password_input.value
    username = username_input.value
    password = password_input.value
    with config_success_message:
        config_success_message.clear_output()
        print("Credentials submitted successfully!")
    submit_button.disabled = True

submit_button.on_click(submit_button_clicked)
display(domain_input, username_input, password_input, submit_button, config_success_message)

# load build
display(widgets.HTML("<h3>Build Ezmeral</h3>"))
build_button = widgets.Button(description='Build')
display(build_button)

progressBar = widgets.IntProgress(
    value=0,
    min=0,
    max=11,
    #description='Build:',
    style={'bar_color': '#1cc88a'},
    orientation='horizontal',
    layout={'width': '1000px'}
)
display(progressBar)


# 01.create-vectorstore

stepVsCreate = widgets.Label(value="01.create-vectorstore")
stepVsCreateStatus = widgets.Output()
with stepVsCreateStatus:
    stepVsCreateStatus.clear_output()
    display(ready_image)
display(widgets.Box([stepVsCreateStatus,stepVsCreate]))

stepVsLoadDoc = widgets.Label(value="Load the Documents")
stepVsLoadDoc.add_class("border-left-ready").add_class("substep")
display(stepVsLoadDoc)

stepVsProcessingDoc = widgets.Label(value="Document Processing: Chunking Text for the Language Model")
stepVsProcessingDoc.add_class("border-left-ready").add_class("substep")
display(stepVsProcessingDoc)

stepVsEmbedding = widgets.Label(value="Generating Embeddings")
stepVsEmbedding.add_class("border-left-ready").add_class("substep")
display(stepVsEmbedding)

stepVsStoringChroma = widgets.Label(value="Storing them in Chroma")
stepVsStoringChroma.add_class("border-left-ready").add_class("substep")
display(stepVsStoringChroma)

# 02.serve-vectorstore
stepVsServce = widgets.Label(value="02.serve-vectorstore")
stepVsServceStatus = widgets.Output()
with stepVsServceStatus:
    stepVsServceStatus.clear_output()
    display(ready_image)
display(widgets.Box([stepVsServceStatus,stepVsServce]))


stepVsLogging = widgets.Label(value="Logging the Vector Store as an Artifact")
stepVsLogging.add_class("border-left-ready").add_class("substep")
display(stepVsLogging)

stepVsInferenceService = widgets.Label(value="Creating the Inference Service")
stepVsInferenceService.add_class("border-left-ready").add_class("substep")
display(stepVsInferenceService)

# 03.serve-llm 
stepLlmServce = widgets.Label(value="03.serve-llm")
stepLlmServceStatus = widgets.Output()
with stepLlmServceStatus:
    stepLlmServceStatus.clear_output()
    display(ready_image)
display(widgets.Box([stepLlmServceStatus,stepLlmServce]))



stepLlmInferenceService = widgets.Label(value="Creating the Inference Service")
stepLlmInferenceService.add_class("border-left-ready").add_class("substep")
display(stepLlmInferenceService)

def build(b):
    # 01.create-vectorstore
    #create_vectorstore();
    
    # 02.serve-vectorstore
    #serve_vectorstore()
    
    # 03.serve-llm 
    serve_llm()
    
    
build_button.on_click(build)


def sucessStep(step_label, progress_bar, incress_progress = 1):
    step_label.remove_class("border-left-ready")
    step_label.add_class("border-left-success")
    progress_bar.value = progress_bar.value + incress_progress
    
def setProgress(status, status_output):
    with status_output:
        status_output.clear_output()
        if status == "loading":
            display(loading_image) 
        elif status == "sucess":
            display(sucess_image) 
            
def create_vectorstore():
    setProgress("loading", stepVsCreateStatus)
    %run ./01.create-vectorstore-new.ipynb
    
    docs = load_docs("documents")
    sucessStep(stepVsLoadDoc, progressBar)
    
    texts = process_docs(docs, chunk_size=500, chunk_overlap=0)
    sucessStep(stepVsProcessingDoc, progressBar)
    
    embeddings = hugging_face_embeddings()
    sucessStep(stepVsEmbedding, progressBar)
    
    db = storing_chroma(texts, embeddings)
    sucessStep(stepVsStoringChroma, progressBar, 2)
    
    #test_query(db)
    setProgress("sucess", stepVsCreateStatus)
    
def serve_vectorstore():
    setProgress("loading", stepVsServceStatus)
    %run ./02.serve-vectorstore-new.ipynb
    
    uri = logging_vector_store_artifact()
    sucessStep(stepVsLogging, progressBar)
    
    apply_vectorstore(uri)
    wait_for_service("vectorstore")
    sucessStep(stepVsInferenceService, progressBar, 3)
    
    setProgress("sucess", stepVsServceStatus)

def serve_llm():
    setProgress("loading", stepLlmServceStatus)
    %run ./04.serve-llm-new.ipynb
    
    apply_llm()
    wait_for_service("llm")
    sucessStep(stepLlmInferenceService, progressBar, 3)
    
    setProgress("sucess", stepLlmServceStatus)
    
def check_inferenceservice_status(name):
    result = subprocess.run(["kubectl", "get", "inferenceservice/"+name, "-n", username],  capture_output=True)
    output = result.stdout.decode("utf-8")
    if not name in output:
        return False

    inference_service = output.split("\n")[1]
    if "True" in inference_service:
        return True
        
def wait_for_service(name): # 5 * 12 * 10 = 10M
    complete = False
    for i in range(12 * 10):
        complete = check_inferenceservice_status(name)
        if complete:
            break
        time.sleep(5)
        
    if(complete):
        print("Complete: " + name)
    else:
        print("Timeout: " + name)
        
